<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/3%EC%9E%A5_%EC%9C%84%EC%84%B1%EA%B8%B0%EB%B0%98_%EC%8B%9D%EC%83%9D%EB%B3%80%ED%99%94_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3장. 위성기반 식생변화 분석

## 3.1. 설악산국립공원 식생지수 분석

In [4]:
import ee
import geemap
import pandas as pd
from ipyleaflet import TileLayer

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

### 3.1.1. 설악산국립공원 경계 추출

In [2]:
# 설악산국립공원 경계 추출
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
seoraksan = wdpa.filter(ee.Filter.eq('WDPAID', 768))

# 선택된 보호지역 이름 확인
wdpa_name = seoraksan.first().get('NAME').getInfo()
print("Name:", wdpa_name)

Name: Seoraksan


In [5]:
# Vworld 배경지도 객체
vworld_base = TileLayer(
    url='https://xdworld.vworld.kr/2d/Base/service/{z}/{x}/{y}.png',
    name='Vworld Base', attribution='Vworld',
)

# 설악산국립공원 경계
m = geemap.Map(width="800px", height="500px")
m.add_layer(vworld_base)
m.addLayer(seoraksan, {'color': 'green'}, wdpa_name)
m.centerObject(seoraksan, 11)
m

Map(center=[38.13576002082187, 128.41324042116062], controls=(WidgetControl(options=['position', 'transparent_…

### 3.1.2. Sentinel-2 이미지 선택 및 필터링

In [ ]:
def mask_s2_clouds(image):
    # QA(Quality Assurance) 밴드 사용, S2에서 구름 마스킹
    qa = image.select('QA60')

    # 비트 10은 구름(clouds), 11은 성층운(cirrus)
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # 구름과 성층운이 0이면 맑은 상태로 간주함.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    return image.updateMask(mask).divide(10000) # 스케일링

Sentinel-2 위성 이미지를 선택하고 필터링하는 과정을 수행합니다. 구체적으로, 2024년 1월 1일부터 1월 31일까지의 기간에 촬영된 이미지 중에서, 지정된 `seoraksan` 지역을 포함하는 이미지를 필터링합니다. 그 다음, 구름이 차지하는 픽셀의 비율이 5% 미만인 이미지만을 필터링하여 선택합니다. 마지막으로, `mask_s2_clouds` 함수를 사용하여 각 이미지에서 구름과 성층운을 마스킹하여 맑은 상태의 이미지만을 남깁니다. 이 과정을 통해 구름이 거의 없는 맑은 상태의 이미지들만을 추출하여 분석이나 가시화에 사용할 수 있는 이미지 컬렉션을 생성합니다.

In [ ]:
# Sentinel-2 이미지 선택 및 필터링
s2_images = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2024-01-01", "2024-01-31")
    .filterBounds(seoraksan)
    # 구름이 5% 미만인 이미지 필터링
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
    .map(mask_s2_clouds)
)

필터링된 이미지의 총 개수는 4장입니다.

In [ ]:
# 이미지 컬렉션의 이미지 개수 확인
image_count = s2_images.size()

# 이미지 개수 출력
print("Image count:", image_count.getInfo())

Image count: 4


필터링된 이미지 컬렉션 내의 모든 이미지들로부터 밴드별 중간값을 계산하여 새로운 단일 이미지를 생성합니다. 이는 구름이 적은 여러 날짜의 이미지들을 통합하여 더 깨끗한 대표 이미지를 얻기 위해 사용됩니다. 이미지는 RGB 컬러로 표시합니다. `B4`, `B3`, `B2`는 각각 적색, 녹색, 청색 색상에 해당하는 Sentinel-2의 밴드입니다.

In [ ]:
# 중간값 이미지 계산
s2_image = s2_images.median()

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map(width="800px", height="500px")
m.add_layer(s2_image, visualization, 'RGB')
m.centerObject(seoraksan, 11) # 지도의 중심 설정
m # 지도 객체 출력

Map(center=[38.13576013377399, 128.41324038484834], controls=(WidgetControl(options=['position', 'transparent_…

## 국립공원 경계 내에서 NDVI 계산 및 통계치 산출하기

프리즘을 통해 볼 수 있듯이, 태양광 스펙트럼은 많은 다른 파장으로 구성되어 있습니다. 태양광이 물체에 비추어질 때, 특정 파장은 흡수되고 다른 파장은 반사됩니다. 식물 잎의 색소인 클로로필(chlorophyll)은 광합성에 사용되는 가시광선을 강하게 흡수합니다. 반면에, 잎의 세포 구조는 근적외선을 강하게 반사합니다. 나무가 클로로필과 클로로필을 포함하는 잎을 더 많이 가질수록, 이러한 파장의 빛은 더 많이 영향을 받습니다. 과학자들은 식물이 빛과 상호작용하는 이 지식을 활용하여 지구 표면 전역의 식물이 흡수하고 반사하는 적색과 근적외선의 파장을 측정하기 위해 위성 센서를 설계함으로써 지구의 풍경을 통틀어 녹색 식생 밀도를 매핑합니다.

식물이 반사하는 적색 빛의 반사율을 근적외선 빛의 반사율에서 빼고, 그 차이를 적색과 근적외선 빛의 반사율의 합으로 나누면 과학자들이 [정규식생지수(NDVI: Normalized Difference Vegetation Index)](https://mynasadata.larc.nasa.gov/mini-lessonactivity/computing-vegetation-health)라고 부르는 값을 얻을 수 있습니다.

아래 코드는 Sentinel-2 위성 이미지를 사용하여 NDVI를 계산하고, 계산된 NDVI 값을 색상 팔레트에 따라 시각화하여 지도에 표시하는 과정을 수행합니다. NDVI는 Sentinel-2 이미지의 B8 밴드(NIR)와 B4 밴드(Red)를 사용하여 계산합니다.

In [ ]:
# NDVI 계산: (NIR - Red) / (NIR + Red)
ndvi = s2_image.normalizedDifference(['B8', 'B4'])

# NDVI 색상 팔레트 정의
ndvi_palette = [
    'FE8374',  # 낮은 NDVI - 갈색
    'FED976',  # 낮은-중간 NDVI - 밝은 녹색
    'CAE23C',  # 중간 NDVI - 녹색
    '98B718',  # 중간-높은 NDVI - 진한 녹색
    '059033',  # 높은 NDVI - 매우 진한 녹색
]

# Vworld 하이브리드지도 객체
vworld_hybrid = TileLayer(
    url='https://xdworld.vworld.kr/2d/Hybrid/service/{z}/{x}/{y}.png',
    name='Vworld Hybrid',
    attribution='Vworld',
)

m = geemap.Map(width="800px", height="500px")
m.add_layer(ndvi, {'min': 0, 'max': 0.5, 'palette': ndvi_palette}, 'NDVI')
m.add_layer(vworld_hybrid)
m.centerObject(seoraksan, 11) # 지도의 중심 설정
m # 지도 객체 출력

Map(center=[38.13576013377399, 128.41324038484834], controls=(WidgetControl(options=['position', 'transparent_…

마지막으로, Sentinel-2 이미지를 기반으로 계산된 NDVI에 대한 다양한 통계치를 계산하고, 이를 데이터프레임으로 변환하여 CSV 파일로 저장하는 과정을 수행합니다.

`reduceRegion` 메서드를 사용하여 지정된 지역(`seoraksan.geometry()`)에서 NDVI의 최소값, 평균값, 중간값, 최대값, 표준편차를 계산합니다. 이는 `ee.Reducer` 객체를 사용하여 여러 통계치를 결합함으로써 한 번의 연산으로 여러 통계치를 얻습니다. `scale` 파라미터는 해상도를 10m로 설정하며, `maxPixels` 파라미터는 처리할 최대 픽셀 수(1e9는 10억 개)를 지정합니다.

In [ ]:
# NDVI 통계치 계산 (최소값, 평균, 중간값, 최대값, 표준편차)
stats = ndvi.reduceRegion(
    reducer=ee.Reducer.min()
    .combine(reducer2=ee.Reducer.mean(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.median(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.max(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True),
    geometry=seoraksan.geometry(),
    scale=10,
    maxPixels=1e9,
)

# 통계치 결과를 DataFrame으로 변환
df_stats = pd.DataFrame(
    [stats.getInfo()],
    columns=["nd_min", "nd_mean", "nd_median", "nd_max", "nd_stdDev"],
    index=["Seoraksan"],
)
df_stats.columns = ["Min", "Mean", "Median", "Max", "StdDev"]

# DataFrame을 CSV 파일로 저장하기
df_stats.to_csv('df_stats.csv', index=True)

# NDVI 통계치 출력
print(df_stats)

                Min      Mean    Median  Max    StdDev
Seoraksan -0.352342  0.237857  0.183589    1  0.197172
